In [1]:
base_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_prediction/"

train_positive_pairs = sc.textFile(base_path + "new_positive_pairs_features/day*/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))
train_unlabled_pairs = sc.textFile(base_path + "new_unlabled_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))

test_positive_pairs = sc.textFile(base_path + "test_positive_pairs/part-*")

# test_pairs_features = sc.textFile(base_path + "new_test_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))
test_pairs_features_50w = sc.textFile(base_path + "test_pairs_features_50w/part-*").map(lambda x: eval(x))

In [9]:
train_positive_pairs.count()

235586

In [10]:
train_unlabled_pairs.count()

8268837

In [2]:
#test_pairs_features_50w = sc.parallelize(test_pairs_features.takeSample(False, 500000))
#test_pairs_features_50w.saveAsTextFile(base_path + "test_pairs_features_50w")


In [17]:
from pyspark.sql import Row
from pyspark.sql.functions import col 
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import RandomForestClassifier

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
import time

In [3]:
test_positive_pairs_list = test_positive_pairs.collect()   # 第四天数据中的正例对
test_data_4thday = test_pairs_features_50w.map(lambda x: Row(label=1.0 if x[0] in test_positive_pairs_list else 0.0,\
                                                         features=Vectors.dense(x[1]))).toDF()

In [20]:
test_data_4thday.count()

500000

In [23]:
y_label_noPU = [i.label for i in test_data_4thday.select('label').collect()]

In [24]:
len(y_label_noPU)

500000

In [9]:
with open('qq_noPU_alfa_v2.log', 'a+') as log_file:
    for alfa in [0.05]: #, 0.10, 0.15, 0.20, 0.25]: #, 0.30, 0.35]: #0.05]:# 
        log_file.write('===========predict whitout PU=========== \n')
        log_file.write(time.asctime() + '\n')
        log_file.write( "==========alfa: " + str(alfa) + "============\n")
        print(time.asctime())
        print("alfa: " + str(alfa))
        # train_unlabeled -> train_neg
        #train_negative_pairs = train_unlabled_pairs.takeSample(False, int(alfa * 8268837))
        #sc.parallelize(train_negative_pairs).coalesce(20).saveAsTextFile(base_path + "wbb_train_negative_pairs-" + str(alfa))
        train_negative_pairs = sc.textFile(base_path + "wbb_train_negative_pairs-" + str(alfa)).map(lambda x: eval(x))

        #log_file.write("length of train_negative_pairs: " + str(len(train_negative_pairs)) + '\n')
        print(time.asctime())
        log_file.write(time.asctime() + '\n')

        #============ without PU =======================
        # predict without PU
        """train_data_noPU = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union((sc.parallelize(train_negative_pairs).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1]))))).toDF()
"""
        train_data_noPU = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(sc.parallelize(train_negative_pairs.takeSample(False, int(230000)))\
               .map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

        #y_label_noPU = [i.label for i in test_data_4thday.select('label').collect()]

        print(time.asctime())
        log_file.write(time.asctime() + '\n')
        
        
        ### DecisionTree
        DT_noPU = DecisionTreeClassifier().fit(train_data_noPU)
        DT_predict_4th_day_noPU = DT_noPU.transform(test_data_4thday.select('features')).join(test_data_4thday, 'features')

        y_label_noPU_DT = [i.label for i in DT_predict_4th_day_noPU.select('label').collect()]
        y_predict_noPU_DT = [i.prediction for i in DT_predict_4th_day_noPU.select('prediction').collect()]

        recall_score_noPU_DT = recall_score(y_label_noPU_DT, y_predict_noPU_DT)
        precision_score_noPU_DT = precision_score(y_label_noPU_DT, y_predict_noPU_DT)
        f1_score_noPU_DT = f1_score(y_label_noPU_DT, y_predict_noPU_DT)
        
        print(recall_score_noPU_DT, precision_score_noPU_DT, f1_score_noPU_DT)

        log_file.write('\n')
        log_file.write("recall_score_noPU_DT: " + str(recall_score_noPU_DT) + '\n')
        log_file.write("precision_score_noPU_DT: " + str(precision_score_noPU_DT) + '\n')
        log_file.write("f1_score_noPU_DT: " + str(f1_score_noPU_DT) + '\n')
        log_file.write(time.asctime() + '\n')
        log_file.write('\n')

        print(time.asctime())
        
        ### NB
        NB_noPU = NaiveBayes().fit(train_data_noPU)
        NB_predict_4th_day_noPU = NB_noPU.transform(test_data_4thday.select('features')).join(test_data_4thday, 'features')


        y_label_noPU_NB = [i.label for i in NB_predict_4th_day_noPU.select('label').collect()]
        y_predict_noPU_NB = [i.prediction for i in NB_predict_4th_day_noPU.select('prediction').collect()]

        recall_score_noPU_NB = recall_score(y_label_noPU_NB, y_predict_noPU_NB)
        precision_score_noPU_NB = precision_score(y_label_noPU_NB, y_predict_noPU_NB)
        f1_score_noPU_NB = f1_score(y_label_noPU_NB, y_predict_noPU_NB)
        
        print(recall_score_noPU_NB, precision_score_noPU_NB, f1_score_noPU_NB)

        log_file.write('\n')
        log_file.write("recall_score_noPU_NB: " + str(recall_score_noPU_NB) + '\n')
        log_file.write("precision_score_noPU_NB: " + str(precision_score_noPU_NB) + '\n')
        log_file.write("f1_score_noPU_NB: " + str(f1_score_noPU_NB) + '\n')
        log_file.write(time.asctime() + '\n')
        log_file.write('\n')

        print(time.asctime())
        
        ### LR
        LR_noPU = LogisticRegression(maxIter=15, regParam=0.04).fit(train_data_noPU)
        LR_predict_4th_day_noPU = LR_noPU.transform(test_data_4thday.select('features')).join(test_data_4thday, 'features')


        y_label_noPU_LR = [i.label for i in LR_predict_4th_day_noPU.select('label').collect()]
        y_predict_noPU_LR = [i.prediction for i in LR_predict_4th_day_noPU.select('prediction').collect()]

        recall_score_noPU_LR = recall_score(y_label_noPU_LR, y_predict_noPU_LR)
        precision_score_noPU_LR = precision_score(y_label_noPU_LR, y_predict_noPU_LR)
        f1_score_noPU_LR = f1_score(y_label_noPU_LR, y_predict_noPU_LR)
        
        print(recall_score_noPU_LR, precision_score_noPU_LR, f1_score_noPU_LR)

        log_file.write('\n')
        log_file.write("recall_score_noPU_LR: " + str(recall_score_noPU_LR) + '\n')
        log_file.write("precision_score_noPU_LR: " + str(precision_score_noPU_LR) + '\n')
        log_file.write("f1_score_noPU_LR: " + str(f1_score_noPU_LR) + '\n')
        log_file.write(time.asctime() + '\n')
        log_file.write('\n')
        

        print(time.asctime())

Thu Feb 16 13:41:29 2017
alfa: 0.05
Thu Feb 16 13:41:29 2017
Thu Feb 16 13:42:12 2017
0.997152869032 0.0149855016177 0.0295272590493
Thu Feb 16 14:00:00 2017
0.770915462111 0.000495872006419 0.000991106508473
Thu Feb 16 14:16:33 2017
0.975689881735 0.000868005323766 0.00173446760896
Thu Feb 16 14:33:25 2017


In [43]:
train_data_noPU.count()

465586

In [21]:
DT_predict_4th_day_noPU.count()

7114542

In [8]:
DT_predict_4th_day_noPU.show(3)

+--------------------+--------------+-----------+----------+-----+
|            features| rawPrediction|probability|prediction|label|
+--------------------+--------------+-----------+----------+-----+
|[0.0,0.0,0.0,0.0,...|[0.0,221037.0]|  [0.0,1.0]|       1.0|  0.0|
|[0.0,0.0,0.0,0.0,...|[0.0,221037.0]|  [0.0,1.0]|       1.0|  0.0|
|[0.0,0.0,0.0,0.0,...|[0.0,221037.0]|  [0.0,1.0]|       1.0|  0.0|
+--------------------+--------------+-----------+----------+-----+
only showing top 3 rows



In [19]:
print(len(y_label_noPU_DT), len(y_predict_noPU_DT))
print(y_label_noPU_DT[:10], y_predict_noPU_DT[:10])

7114542 7114542
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [44]:
print(time.asctime())
DT_noPU = DecisionTreeClassifier(maxDepth=20).fit(train_data_noPU)
DT_predict_4th_day_noPU = DT_noPU.transform(test_data_4thday.select('features')) # .join(test_data_4thday, 'features')

# y_label_noPU_DT = [i.label for i in DT_predict_4th_day_noPU.select('label').collect()]
y_predict_noPU_DT = [i.prediction for i in DT_predict_4th_day_noPU.select('prediction').collect()]
print(len(y_predict_noPU_DT))

recall_score_noPU_DT = recall_score(y_label_noPU, y_predict_noPU_DT)
precision_score_noPU_DT = precision_score(y_label_noPU, y_predict_noPU_DT)
f1_score_noPU_DT = f1_score(y_label_noPU, y_predict_noPU_DT)

print(recall_score_noPU_DT, precision_score_noPU_DT, f1_score_noPU_DT)
print(time.asctime())

Thu Feb 16 17:08:13 2017
500000
0.996933858958 0.0149906966788 0.0295372476243
Thu Feb 16 17:13:36 2017


In [28]:
recall_score_noPU_DT = recall_score(y_label_noPU, y_predict_noPU_DT)
precision_score_noPU_DT = precision_score(y_label_noPU, y_predict_noPU_DT)
f1_score_noPU_DT = f1_score(y_label_noPU, y_predict_noPU_DT)

print(recall_score_noPU_DT, precision_score_noPU_DT, f1_score_noPU_DT)
print(time.asctime())

0.996933858958 0.0149906966788 0.0295372476243
Thu Feb 16 16:44:05 2017


In [29]:
print(len(y_label_noPU), len(y_predict_noPU_DT))
print(y_label_noPU[:10], y_predict_noPU_DT[:10])

500000 500000
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0]


In [45]:
tp = 0
fn = 0
fp = 0
tn = 0
for i in range(500000):
    if y_label_noPU[i] == 1.0 and y_predict_noPU_DT[i] == 1.0:
        tp += 1
    if y_label_noPU[i] == 1.0 and y_predict_noPU_DT[i] == 0.0:
        fn += 1
    if y_label_noPU[i] == 0.0 and y_predict_noPU_DT[i] == 1.0:
        fp += 1
    if y_label_noPU[i] == 0.0 and y_predict_noPU_DT[i] == 0.0:
        tn += 1

In [46]:
print(tp, fn, fp, tn)

4552 14 299103 196331


In [47]:
(196331+4552)/500000

0.401766

In [32]:
4552/(4552+299103)

0.01499069667879666

In [41]:
k = 0
for i in DT_predict_4th_day_noPU.take(500000):
    if i.features[-1] == 0.0 and i.features[-2] == 0.0 and i.prediction == 1.0:
        #print(i.features, i.prediction)
        k += 1
print(k)

1051


In [42]:
print(k)

1051


In [55]:
print(time.asctime())
NB_noPU = NaiveBayes().fit(train_data_noPU)
NB_predict_4th_day_noPU = NB_noPU.transform(test_data_4thday.select('features'))


#y_label_noPU_NB = [i.label for i in NB_predict_4th_day_noPU.select('label').collect()]
y_predict_noPU_NB = [i.prediction for i in NB_predict_4th_day_noPU.select('prediction').collect()]

recall_score_noPU_NB = recall_score(y_label_noPU, y_predict_noPU_NB)
precision_score_noPU_NB = precision_score(y_label_noPU, y_predict_noPU_NB)
f1_score_noPU_NB = f1_score(y_label_noPU, y_predict_noPU_NB)
tp_NB = 0
tn_NB = 0
for i in range(500000):
    if y_label_noPU[i] == 1.0 and y_predict_noPU_NB[i] == 1.0:
        tp_NB += 1
    if y_label_noPU[i] == 0.0 and y_predict_noPU_NB[i] == 0.0:
        tn_NB += 1
accuracy_NB = (tp_NB + tn_NB)/500000

print(recall_score_noPU_NB, precision_score_noPU_NB, f1_score_noPU_NB, accuracy_NB)
print(time.asctime())

Thu Feb 16 17:43:00 2017
0.770915462111 0.00726123531766 0.0143869601825 0.035416
Thu Feb 16 17:47:24 2017


In [51]:
tp_NB = 0
tn_NB = 0
for i in range(500000):
    if y_label_noPU[i] == 1.0 and y_predict_noPU_NB[i] == 1.0:
        tp_NB += 1
    if y_label_noPU[i] == 0.0 and y_predict_noPU_NB[i] == 0.0:
        tn_NB += 1
        
accuracy_NB = (tp_NB + tn_NB)/500000
print(accuracy_NB)

0.035416


In [54]:
print(tp_NB, tn_NB)

3520 14188


In [49]:
print(time.asctime())
RF_noPU = RandomForestClassifier(numTrees=3, maxDepth=8).fit(train_data_noPU)
RF_predict_4th_day_noPU = RF_noPU.transform(test_data_4thday.select('features'))


#y_label_noPU_RF = [i.label for i in RF_predict_4th_day_noPU.select('label').collect()]
y_predict_noPU_RF = [i.prediction for i in RF_predict_4th_day_noPU.select('prediction').collect()]

recall_score_noPU_RF = recall_score(y_label_noPU, y_predict_noPU_RF)
precision_score_noPU_RF = precision_score(y_label_noPU, y_predict_noPU_RF)
f1_score_noPU_RF = f1_score(y_label_noPU, y_predict_noPU_RF)
tp_RF = 0
tn_RF = 0
for i in range(500000):
    if y_label_noPU[i] == 1.0 and y_predict_noPU_RF[i] == 1.0:
        tp_RF += 1
    if y_label_noPU[i] == 0.0 and y_predict_noPU_RF[i] == 0.0:
        tn_RF += 1
accuracy_RF = (tp_RF + tn_RF)/500000

print(recall_score_noPU_RF, precision_score_noPU_RF, f1_score_noPU_RF, accuracy_RF)
print(time.asctime())

Thu Feb 16 17:30:15 2017
0.962768287341 0.0352681615789 0.0680437424058 0.401766
Thu Feb 16 17:35:39 2017


In [53]:
tp_RF = 0
tn_RF = 0
for i in range(500000):
    if y_label_noPU[i] == 1.0 and y_predict_noPU_RF[i] == 1.0:
        tp_RF += 1
    if y_label_noPU[i] == 0.0 and y_predict_noPU_RF[i] == 0.0:
        tn_RF += 1
accuracy_RF = (tp_RF + tn_RF)/500000
print(accuracy_RF)

0.759162


In [50]:
print(time.asctime())
LR_noPU = LogisticRegression(maxIter=100, regParam=0.0).fit(train_data_noPU)
LR_predict_4th_day_noPU = LR_noPU.transform(test_data_4thday.select('features'))


#y_label_noPU_LR = [i.label for i in LR_predict_4th_day_noPU.select('label').collect()]
y_predict_noPU_LR = [i.prediction for i in LR_predict_4th_day_noPU.select('prediction').collect()]

recall_score_noPU_LR = recall_score(y_label_noPU, y_predict_noPU_LR)
precision_score_noPU_LR = precision_score(y_label_noPU, y_predict_noPU_LR)
f1_score_noPU_LR = f1_score(y_label_noPU, y_predict_noPU_LR)
tp_LR = 0
tn_LR = 0
for i in range(500000):
    if y_label_noPU[i] == 1.0 and y_predict_noPU_LR[i] == 1.0:
        tp_LR += 1
    if y_label_noPU[i] == 0.0 and y_predict_noPU_LR[i] == 0.0:
        tn_LR += 1
accuracy_LR = (tp_LR + tn_LR)/500000

print(recall_score_noPU_LR, precision_score_noPU_LR, f1_score_noPU_LR, accuracy_LR)
print(time.asctime())

Thu Feb 16 17:35:39 2017
0.975689881735 0.012053864948 0.0238135328218 0.401766
Thu Feb 16 17:39:48 2017


In [52]:
tp_LR = 0
tn_LR = 0
for i in range(500000):
    if y_label_noPU[i] == 1.0 and y_predict_noPU_LR[i] == 1.0:
        tp_LR += 1
    if y_label_noPU[i] == 0.0 and y_predict_noPU_LR[i] == 0.0:
        tn_LR += 1
accuracy_LR = (tp_LR + tn_LR)/500000
print(accuracy_LR)

0.269506


In [15]:
LR_accuracy_noPU = (LR_predict_4th_day_noPU.filter(LR_predict_4th_day_noPU.prediction == LR_predict_4th_day_noPU.label).count())\
/(LR_predict_4th_day_noPU.count())

In [16]:
print(LR_accuracy_noPU)

0.9462186040928566


In [42]:
time.asctime()

'Tue Feb 14 16:27:17 2017'

In [88]:
aa = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,3.0,0.0,0.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,13.0,64.0,70.0,31.0,8.0,0.0,0.0,42.0,222.0,2109.0,2055.0,-5847.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,16.0,19.0,12087.0]

In [89]:
len(aa)

194

In [90]:
aa.index(-5847.0)

109

In [56]:
a = [('aa',[0,1,2,3,0]), ('aa',[0,1,2,4,0]), ('aasas',[10,1,1,0,0])]
c = [('abs',[0,0,0,0,0]), ('aas',[0,1,1,0,0]), ('aaas',[110,1111,1,0,0])]

In [59]:
b = sc.parallelize(a).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1]))).union((sc.parallelize(c).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1]))))).toDF()

In [60]:
b.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.0,1.0,2.0,3.0,...|  0.0|
|[0.0,1.0,2.0,4.0,...|  0.0|
|[10.0,1.0,1.0,0.0...|  0.0|
|[0.0,0.0,0.0,0.0,...|  1.0|
|[0.0,1.0,1.0,0.0,...|  1.0|
|[110.0,1111.0,1.0...|  1.0|
+--------------------+-----+



In [80]:
sd = sc.parallelize([[0,1,2,4,0], [0,0,0,0,0], [0,1,2,3,0], [110,1111,1,0,0]]).collect()
sd = [Vectors.dense(x) for x in sd]

In [81]:
bb = b.rdd.filter(lambda x: x.features in sd and x.label == 0.0)

In [84]:
bb.take(4)

[Row(features=DenseVector([0.0, 1.0, 2.0, 3.0, 0.0]), label=0.0),
 Row(features=DenseVector([0.0, 1.0, 2.0, 4.0, 0.0]), label=0.0)]

In [88]:
bb.map(lambda x: x.label).collect()

[0.0, 0.0]

In [82]:
xx = [i.label for i in b.select('label').collect()]

In [83]:
xx

[0.0, 0.0, 1.0, 1.0]

In [79]:
aaa = b.select('label').collect()

In [80]:
for i in aaa:
    print(i.label)

0.0
0.0
1.0
1.0


In [47]:
m = NaiveBayes().fit(b)

In [48]:
test1 = sc.parallelize([Row(features=Vectors.dense([0,0,2,3,0]))]).toDF()

In [81]:
m.transform(b).show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[0.0,1.0,2.0,3.0,...|  0.0|[-7.4795649893611...|[0.94319247350861...|       0.0|
|[0.0,1.0,2.0,4.0,...|  0.0|[-8.2904952055774...|[0.98100834840376...|       0.0|
|[0.0,0.0,0.0,0.0,...|  1.0|[-0.6931471805599...|           [0.5,0.5]|       0.0|
|[0.0,1.0,1.0,0.0,...|  1.0|[-3.7658404952500...|[0.36189069423929...|       1.0|
+--------------------+-----+--------------------+--------------------+----------+



In [50]:
mm = GBTClassifier().fit(b)

In [51]:
mm.transform(test1).head()

Row(features=DenseVector([0.0, 0.0, 2.0, 3.0, 0.0]), prediction=0.0)

In [52]:
mdt = DecisionTreeClassifier().fit(b)

In [56]:
cs = mdt.transform(test1)

In [57]:
cs.show()

+--------------------+-------------+-----------+----------+
|            features|rawPrediction|probability|prediction|
+--------------------+-------------+-----------+----------+
|[0.0,0.0,2.0,3.0,...|    [2.0,0.0]|  [1.0,0.0]|       0.0|
+--------------------+-------------+-----------+----------+



In [62]:
cs.select('prediction').collect()

[Row(prediction=0.0)]

In [43]:
cc = [('day4-8615195898003-14.17.19.77', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 3, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5], [12], [26], [41], [39064], [169], [0, 0, 0, 0, 0, 0, 62, 0, 179, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 179, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8, 5, 2, 0, 1, 1, 3, 7, 8, 14, 9, 11, 10, 11, 13, 12, 9, 11, 11, 13, 15, 16, 27, 24], [241], [1710], [1328], [2603900]]), ('day4-8615195898003-111.10.27.180', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 3, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5], [12], [26], [41], [39064], [3], [0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0], [5], [409], [620], [89159]]), ('day4-8615195898003-111.30.132.109', [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 3, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5], [12], [26], [41], [39064], [3], [0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4], [29], [27], [27390]])]

In [86]:
with open('qq11.log', 'a+') as lf:
    lf.write("sdfsdfhasj")
    lf.write("=======")